In [2]:
import d3rlpy
from d3rlpy.algos import COMBO
from sklearn.model_selection import train_test_split
import gym
import numpy as np

## Params

In [3]:
d3rlpy.seed(1)
use_gpu = True
# prepare environment
env = gym.make("InvertedPendulum-v4")
eval_env = gym.make("InvertedPendulum-v4")
env.reset(seed=1)
eval_env.reset(seed=1)

/home/yasin/.local/share/virtualenvs/symmetry-cs285-U_vq4NrE/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:69: UserWarning: WARN: Agent's minimum action space value is -infinity. This is probably too low.
  logger.warn(
/home/yasin/.local/share/virtualenvs/symmetry-cs285-U_vq4NrE/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:73: UserWarning: WARN: Agent's maximum action space value is infinity. This is probably too high
  logger.warn(
/home/yasin/.local/share/virtualenvs/symmetry-cs285-U_vq4NrE/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:97: UserWarning: WARN: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  logger.warn(


array([ 0.00023643,  0.00900927, -0.00711681,  0.00897299])

## Create dataset

In [5]:
actor_encoder = d3rlpy.models.encoders.DefaultEncoderFactory(dropout_rate=0.2)
# setup algorithm
sac = d3rlpy.algos.SAC(
    batch_size=256,
    actor_encoder_factory=actor_encoder,
    actor_learning_rate=3e-4,
    critic_learning_rate=3e-4,
    temp_learning_rate=3e-4,
    use_gpu=use_gpu
)

# prepare utilities
buffer = d3rlpy.online.buffers.ReplayBuffer(maxlen=1000000, env=env)

# start training
sac.fit_online(
    env,
    buffer,
    eval_env=eval_env,
    n_steps=100000,
    n_steps_per_epoch=1000,
    update_interval=1,
    update_start_step=1000,
    tensorboard_dir='tensorboard_logs'
)

2023-09-29 10:36:45 [info     ] Directory is created at d3rlpy_logs/SAC_online_20230929103645
2023-09-29 10:36:45 [debug    ] Building model...
2023-09-29 10:36:45 [debug    ] Model has been built.
2023-09-29 10:36:45 [info     ] Parameters are saved to d3rlpy_logs/SAC_online_20230929103645/params.json params={'action_scaler': None, 'actor_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': 0.2}}, 'actor_learning_rate': 0.0003, 'actor_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'batch_size': 256, 'critic_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'critic_learning_rate': 0.0003, 'critic_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'gamma': 0.99, 'generated_maxlen': 100000, 'initial_temperature': 1.0, 'n_critics': 2, '

/home/yasin/.local/share/virtualenvs/symmetry-cs285-U_vq4NrE/lib/python3.8/site-packages/gym/core.py:200: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed)` instead.
  deprecation(


  0%|          | 0/100000 [00:00<?, ?it/s]

2023-09-29 10:36:47 [info     ] Model parameters are saved to d3rlpy_logs/SAC_online_20230929103645/model_1000.pt
2023-09-29 10:36:48 [info     ] SAC_online_20230929103645: epoch=1 step=1000 epoch=1 metrics={'time_inference': 0.002146885395050049, 'time_environment_step': 0.00016734671592712403, 'time_step': 0.00239652943611145, 'rollout_return': 9.96, 'evaluation': 14.6} step=1000
2023-09-29 10:37:03 [info     ] Model parameters are saved to d3rlpy_logs/SAC_online_20230929103645/model_2000.pt
2023-09-29 10:37:03 [info     ] SAC_online_20230929103645: epoch=2 step=2000 epoch=2 metrics={'time_inference': 0.0013801386356353759, 'time_environment_step': 0.00013177084922790528, 'time_sample_batch': 0.0003177614212036133, 'time_algorithm_update': 0.012360470056533814, 'temp_loss': 1.3707251897454262, 'temp': 0.8709913108348847, 'critic_loss': 0.6587207683324814, 'actor_loss': -4.27164307051897, 'time_step': 0.014282214879989624, 'rollout_return': 17.24137931034483, 'evaluation': 85.4} step=

KeyboardInterrupt: 

In [6]:
# export replay buffer as MDPDataset
dataset = buffer.to_mdp_dataset()

# save MDPDataset
dataset.dump('d3rlpy_data/inverted_pendulum.h5')

## Load the dataset

In [5]:
dataset = d3rlpy.dataset.MDPDataset.load('d3rlpy_data/inverted_pendulum.h5')

In [6]:
train_episodes, test_episodes = train_test_split(dataset)

## Dynamics learning

In [17]:
dynamics = d3rlpy.dynamics.ProbabilisticEnsembleDynamics(learning_rate=1e-4, use_gpu=True)

In [18]:
# same as algorithms
dynamics.fit(train_episodes,
             eval_episodes=test_episodes,
             n_epochs=100,
             scorers={
                'observation_error': d3rlpy.metrics.scorer.dynamics_observation_prediction_error_scorer,
                'reward_error': d3rlpy.metrics.scorer.dynamics_reward_prediction_error_scorer,
                'variance': d3rlpy.metrics.scorer.dynamics_prediction_variance_scorer,
             },
            tensorboard_dir='tensorboard_logs/dynamics')

2023-09-29 11:11:24 [debug    ] RoundIterator is selected.
2023-09-29 11:11:24 [info     ] Directory is created at d3rlpy_logs/ProbabilisticEnsembleDynamics_20230929111124
2023-09-29 11:11:24 [debug    ] Building models...
2023-09-29 11:11:24 [debug    ] Models have been built.
2023-09-29 11:11:24 [info     ] Parameters are saved to d3rlpy_logs/ProbabilisticEnsembleDynamics_20230929111124/params.json params={'action_scaler': None, 'batch_size': 100, 'discrete_action': False, 'encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'gamma': 1.0, 'generated_maxlen': 100000, 'learning_rate': 0.0001, 'n_ensembles': 5, 'n_frames': 1, 'n_steps': 1, 'optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0.0001, 'amsgrad': False}, 'real_ratio': 1.0, 'reward_scaler': None, 'scaler': None, 'use_gpu': 0, 'variance_type': 'max', 'algorithm': 'ProbabilisticEnsembleDynamics', 'observation_shape': (4,

Epoch 1/100:   0%|          | 0/530 [00:00<?, ?it/s]

2023-09-29 11:11:35 [info     ] ProbabilisticEnsembleDynamics_20230929111124: epoch=1 step=530 epoch=1 metrics={'time_sample_batch': 0.00033259661692493366, 'time_algorithm_update': 0.017799991931555407, 'loss': -20.412133104080016, 'time_step': 0.018235126981195414, 'observation_error': 0.05122452501369622, 'reward_error': 0.0010851811401877767, 'variance': 0.061272936777414716} step=530
2023-09-29 11:11:35 [info     ] Model parameters are saved to d3rlpy_logs/ProbabilisticEnsembleDynamics_20230929111124/model_530.pt


Epoch 2/100:   0%|          | 0/530 [00:00<?, ?it/s]

2023-09-29 11:11:47 [info     ] ProbabilisticEnsembleDynamics_20230929111124: epoch=2 step=1060 epoch=2 metrics={'time_sample_batch': 0.00030880694119435434, 'time_algorithm_update': 0.01719002048924284, 'loss': -32.47664208772048, 'time_step': 0.01759045843808156, 'observation_error': 0.03070271193117635, 'reward_error': 0.0002527767268774024, 'variance': 0.029529351023722516} step=1060
2023-09-29 11:11:47 [info     ] Model parameters are saved to d3rlpy_logs/ProbabilisticEnsembleDynamics_20230929111124/model_1060.pt


...

Epoch 99/100:   0%|          | 0/530 [00:00<?, ?it/s]

2023-09-29 11:28:33 [info     ] ProbabilisticEnsembleDynamics_20230929111124: epoch=99 step=52470 epoch=99 metrics={'time_sample_batch': 0.00025926356045704967, 'time_algorithm_update': 0.012917462384925698, 'loss': -78.50234591646014, 'time_step': 0.013246399951430987, 'observation_error': 0.026967493642121053, 'reward_error': 1.785908835401077e-06, 'variance': 0.012565490767809587} step=52470
2023-09-29 11:28:33 [info     ] Model parameters are saved to d3rlpy_logs/ProbabilisticEnsembleDynamics_20230929111124/model_52470.pt


Epoch 100/100:   0%|          | 0/530 [00:00<?, ?it/s]

2023-09-29 11:28:41 [info     ] ProbabilisticEnsembleDynamics_20230929111124: epoch=100 step=53000 epoch=100 metrics={'time_sample_batch': 0.00026225908747259176, 'time_algorithm_update': 0.01337816355363378, 'loss': -78.91229915978774, 'time_step': 0.013713202836378566, 'observation_error': 0.02558656144630602, 'reward_error': 2.5343300311431827e-06, 'variance': 0.012141100572091912} step=53000
2023-09-29 11:28:41 [info     ] Model parameters are saved to d3rlpy_logs/ProbabilisticEnsembleDynamics_20230929111124/model_53000.pt


[(1,
  {'time_sample_batch': 0.00033259661692493366,
   'time_algorithm_update': 0.017799991931555407,
   'loss': -20.412133104080016,
   'time_step': 0.018235126981195414,
   'observation_error': 0.05122452501369622,
   'reward_error': 0.0010851811401877767,
   'variance': 0.061272936777414716}),
 (2,
  {'time_sample_batch': 0.00030880694119435434,
   'time_algorithm_update': 0.01719002048924284,
   'loss': -32.47664208772048,
   'time_step': 0.01759045843808156,
   'observation_error': 0.03070271193117635,
   'reward_error': 0.0002527767268774024,
   'variance': 0.029529351023722516}),
 (3,
  {'time_sample_batch': 0.0003708915890387769,
   'time_algorithm_update': 0.018562685768559295,
   'loss': -39.59974348859967,
   'time_step': 0.019040496394319355,
   'observation_error': 0.019320719303135253,
   'reward_error': 0.00012813193112252102,
   'variance': 0.008370734354173041}),
 (4,
  {'time_sample_batch': 0.00041189598587324037,
   'time_algorithm_update': 0.02051009232143186,
   '

## Load Dynamics

In [5]:
# load trained dynamics model
dynamics_model_path = "d3rlpy_logs/ProbabilisticEnsembleDynamics_20230929111124"
dynamics = d3rlpy.dynamics.ProbabilisticEnsembleDynamics.from_json(dynamics_model_path + '/params.json')
dynamics.load_model(dynamics_model_path + '/model_53000.pt')

2023-09-29 15:30:26 [warning  ] Unused arguments are passed.   algorithm=ProbabilisticEnsembleDynamics gamma=1.0 generated_maxlen=100000 n_steps=1 real_ratio=1.0


## Train Offline RL Algorithm

In [6]:
encoders = d3rlpy.models.encoders.DefaultEncoderFactory(dropout_rate=0.2)
# give COMBO as the generator argument.
combo = COMBO(dynamics=dynamics, critic_encoder_factory=encoders, actor_encoder_factory=encoders
              use_gpu=use_gpu)

In [ ]:
combo.fit(dataset = train_episodes, eval_episodes=test_episodes, n_steps=100000, n_steps_per_epoch=1000, tensorboard_dir="tensorboard_logs",
         scorers={
            'environment': d3rlpy.metrics.scorer.evaluate_on_environment(eval_env)
        })

2023-09-29 15:30:35 [debug    ] RandomIterator is selected.
2023-09-29 15:30:35 [info     ] Directory is created at d3rlpy_logs/COMBO_20230929153035
2023-09-29 15:30:35 [debug    ] Building models...
2023-09-29 15:30:36 [debug    ] Models have been built.
2023-09-29 15:30:36 [info     ] Parameters are saved to d3rlpy_logs/COMBO_20230929153035/params.json params={'action_scaler': None, 'actor_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': 0.2}}, 'actor_learning_rate': 0.0001, 'actor_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'batch_size': 256, 'conservative_weight': 1.0, 'critic_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': 0.2}}, 'critic_learning_rate': 0.0003, 'critic_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'gamma':

Epoch 1/1000:   0%|          | 0/1000 [00:00<?, ?it/s]

2023-09-29 15:30:51 [debug    ] 250000 transitions are generated. fake_transitions=250000 real_transitions=53262
2023-09-29 15:31:18 [info     ] COMBO_20230929153035: epoch=1 step=1000 epoch=1 metrics={'time_sample_batch': 0.0010948445796966552, 'time_algorithm_update': 0.024626394510269167, 'critic_loss': 7.946767935276031, 'actor_loss': -3.194101221084595, 'temp_loss': 1.4520865498781204, 'temp': 0.9536138231158257, 'time_step': 0.025825464487075806, 'environment': 46.3} step=1000
2023-09-29 15:31:18 [info     ] Model parameters are saved to d3rlpy_logs/COMBO_20230929153035/model_1000.pt


Epoch 2/1000:   0%|          | 0/1000 [00:00<?, ?it/s]

2023-09-29 15:31:35 [debug    ] 250000 transitions are generated. fake_transitions=500000 real_transitions=53262
2023-09-29 15:32:02 [info     ] COMBO_20230929153035: epoch=2 step=2000 epoch=2 metrics={'time_sample_batch': 0.0011157858371734618, 'time_algorithm_update': 0.024792260885238646, 'critic_loss': 7.954225881099701, 'actor_loss': -7.0867501902580265, 'temp_loss': 1.2333806045055389, 'temp': 0.8689545059204101, 'time_step': 0.026010702610015868, 'environment': 69.3} step=2000
2023-09-29 15:32:02 [info     ] Model parameters are saved to d3rlpy_logs/COMBO_20230929153035/model_2000.pt


...

Epoch 52/1000:   0%|          | 0/1000 [00:00<?, ?it/s]

2023-09-29 16:08:44 [debug    ] 250000 transitions are generated. fake_transitions=1250000 real_transitions=53262
2023-09-29 16:09:17 [info     ] COMBO_20230929153035: epoch=52 step=52000 epoch=52 metrics={'time_sample_batch': 0.000996147871017456, 'time_algorithm_update': 0.022960602760314943, 'critic_loss': 21946.27154309082, 'actor_loss': 144.5962454147339, 'temp_loss': -0.12356063885241747, 'temp': 0.34013513389229777, 'time_step': 0.02407371735572815, 'environment': 1000.0} step=52000
2023-09-29 16:09:17 [info     ] Model parameters are saved to d3rlpy_logs/COMBO_20230929153035/model_52000.pt


Epoch 53/1000:   0%|          | 0/1000 [00:00<?, ?it/s]

2023-09-29 16:09:32 [debug    ] 250000 transitions are generated. fake_transitions=1250000 real_transitions=53262
2023-09-29 16:10:02 [info     ] COMBO_20230929153035: epoch=53 step=53000 epoch=53 metrics={'time_sample_batch': 0.0008946323394775391, 'time_algorithm_update': 0.020589677333831786, 'critic_loss': 24185.791935058594, 'actor_loss': 170.99163961791993, 'temp_loss': -0.13503186032548548, 'temp': 0.380317705899477, 'time_step': 0.02157444357872009, 'environment': 1000.0} step=53000
2023-09-29 16:10:02 [info     ] Model parameters are saved to d3rlpy_logs/COMBO_20230929153035/model_53000.pt


Epoch 54/1000:   0%|          | 0/1000 [00:00<?, ?it/s]

## Load the Policy

In [9]:
trained_policy = COMBO()
# initialize with dataset
trained_policy.build_with_dataset(dataset)
# Load entire model parameters.
trained_policy.load_model('d3rlpy_logs/COMBO_20230929153035/model_53000.pt')

## See the policy running

In [10]:
scorer = d3rlpy.metrics.scorer.evaluate_on_environment(eval_env, render=True)
mean_episode_return = scorer(trained_policy)